# MTA BusTime SIRI API demo

## setup

In [1]:
!pip install requests trio asks

In [1]:
import requests

### create a timer decorator (from RealPython)

In [11]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = time.perf_counter()
        value = func(*args, **kwargs)
        toc = time.perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

### request url generators

In [3]:
API_KEY = '088886bd-cc48-4d7c-bd8a-498d353d7d13'

def url_fetch_all():
    return f'http://bustime.mta.info/api/siri/vehicle-monitoring.json?key={API_KEY}&VehicleMonitoringDetailLevel=calls'

def url_fetch_1(route):
    return f'http://bustime.mta.info/api/siri/vehicle-monitoring.json?key={API_KEY}&VehicleMonitoringDetailLevel=calls&LineRef={route}'


# fetch a snapshot of entire system — all buses, right now

In [6]:
@timer
def get_all():
    response = requests.get(url_fetch_all())
    jsonResponse = response.json()
    print(jsonResponse)
    return jsonResponse

buses = get_all()
# ['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
len(buses)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



1

# fetch only ONE route

In [4]:
@ timer
def get_one(route):
    response = requests.get(url_fetch_1(route))
    jsonResponse = response.json()
    return jsonResponse

buses = get_one('M15')['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']
len(buses)

Elapsed time: 0.3977 seconds


15

# fetch entire system one route at a time

## get the list of routes from a sister API, MTA BusTime's One Bus Away

In [5]:
def get_OBA_routelist():
    url = "http://bustime.mta.info/api/where/routes-for-agency/MTA%20NYCT.json?key=" + API_KEY
    response = requests.get(url, timeout=30)
    return [x['id'] for x in response.json()['data']['list']]

print ( get_OBA_routelist() )


['MTA NYCT_M34+', 'MTA NYCT_B52', 'MTA NYCT_Q24', 'MTA NYCT_S66', 'MTA NYCT_Q27', 'MTA NYCT_Q26', 'MTA NYCT_Q28', 'MTA NYCT_B46+', 'MTA NYCT_S61', 'MTA NYCT_SIM15', 'MTA NYCT_S62', 'MTA NYCT_B44', 'MTA NYCT_B43', 'MTA NYCT_SIM10', 'MTA NYCT_B46', 'MTA NYCT_SIM11', 'MTA NYCT_B45', 'MTA NYCT_B48', 'MTA NYCT_B47', 'MTA NYCT_SIM9', 'MTA NYCT_B49', 'MTA NYCT_SIM8', 'MTA NYCT_SIM7', 'MTA NYCT_SIM6', 'MTA NYCT_B60', 'MTA NYCT_SIM5', 'MTA NYCT_SIM4', 'MTA NYCT_B62', 'MTA NYCT_SIM1C', 'MTA NYCT_SIM3', 'MTA NYCT_B61', 'MTA NYCT_SIM2', 'MTA NYCT_B64', 'MTA NYCT_SIM1', 'MTA NYCT_B63', 'MTA NYCT_S76', 'MTA NYCT_Q36', 'MTA NYCT_S78', 'MTA NYCT_S79+', 'MTA NYCT_BX12+', 'MTA NYCT_Q30', 'MTA NYCT_SIM25', 'MTA NYCT_Q32', 'MTA NYCT_S74', 'MTA NYCT_SIM26', 'MTA NYCT_Q31', 'MTA NYCT_B54', 'MTA NYCT_B57', 'MTA NYCT_D99', 'MTA NYCT_SIM22', 'MTA NYCT_BX2', 'MTA NYCT_BX1', 'MTA NYCT_B70', 'MTA NYCT_BX4', 'MTA NYCT_X28', 'MTA NYCT_BX3', 'MTA NYCT_X27', 'MTA NYCT_BX6', 'MTA NYCT_BX5', 'MTA NYCT_B74', 'MTA NYCT_S

## iterate over the list, calling get_one() for each route

In [6]:
def get_one(route): #no timer decorator
    response = requests.get(url_fetch_1(route))
    jsonResponse = response.json()
    return jsonResponse

@ timer
def get_all_one_at_a_time():
    responses = []
    for route in get_OBA_routelist():
        
        jsonResponse = get_one(route)
        try:
            print(
                f"{len(jsonResponse['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'])} buses on {route}"
            )
        except:
            print(f'0 buses on {route}')
        responses.append(jsonResponse)
    return responses

responses = get_all_one_at_a_time()


0 buses on MTA NYCT_M34+
13 buses on MTA NYCT_B52
8 buses on MTA NYCT_Q24
2 buses on MTA NYCT_S66
24 buses on MTA NYCT_Q27
3 buses on MTA NYCT_Q26
4 buses on MTA NYCT_Q28
0 buses on MTA NYCT_B46+
2 buses on MTA NYCT_S61
2 buses on MTA NYCT_SIM15
3 buses on MTA NYCT_S62
13 buses on MTA NYCT_B44
12 buses on MTA NYCT_B43
2 buses on MTA NYCT_SIM10
16 buses on MTA NYCT_B46
1 buses on MTA NYCT_SIM11
5 buses on MTA NYCT_B45
3 buses on MTA NYCT_B48
10 buses on MTA NYCT_B47
2 buses on MTA NYCT_SIM9
12 buses on MTA NYCT_B49
4 buses on MTA NYCT_SIM8
3 buses on MTA NYCT_SIM7
3 buses on MTA NYCT_SIM6
8 buses on MTA NYCT_B60
0 buses on MTA NYCT_SIM5
0 buses on MTA NYCT_SIM4
7 buses on MTA NYCT_B62
7 buses on MTA NYCT_SIM1C
0 buses on MTA NYCT_SIM3
5 buses on MTA NYCT_B61
2 buses on MTA NYCT_SIM2
7 buses on MTA NYCT_B64
2 buses on MTA NYCT_SIM1
11 buses on MTA NYCT_B63
5 buses on MTA NYCT_S76
7 buses on MTA NYCT_Q36
6 buses on MTA NYCT_S78
0 buses on MTA NYCT_S79+
0 buses on MTA NYCT_BX12+
12 buses o

# fetch entire system one route at a time — asynchronously...

In [9]:
API_KEY = '088886bd-cc48-4d7c-bd8a-498d353d7d13'

import requests
from time import time
import datetime as dt
import trio

def async_grab_and_store(localhost, cwd):
    start = time()
    SIRI_request_urlpaths = get_SIRI_request_urlpaths()
    feeds = []

    async def grabber(s,a_path,route_id):
        try:
            r = await s.get(path=a_path, retries=10)
            feeds.append({route_id:r})
        except Exception as e:
            logging.error ('\tCould not fetch feed for {}. (Increase max retries for Session.get()?)'.format(route_id) )

    async def main(path_list):
        from asks.sessions import Session

        if localhost is True:
            s = Session('http://bustime.mta.info', connections=5)
        else:
            s = Session('http://bustime.mta.info', connections=config.config['http_connections'])
        async with trio.open_nursery() as n:
            for path_bundle in path_list:
                for route_id,path in path_bundle.items():
                    n.start_soon(grabber, s, path, route_id )

    trio.run(main, SIRI_request_urlpaths)

    # dump to the various locations
    timestamp = dt.datetime.now()
    # DataLake(cwd).make_puddles(feeds, DatePointer(timestamp))
    # DataStore(cwd).make_barrels(feeds, DatePointer(timestamp))

    # report results to console
    n_buses = num_buses(feeds)
    end = time()
    print('Grabbed {} route feeds containing {} buses in {:2f} seconds at {}.'.format(len(feeds),n_buses,(end - start), dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    return


def get_OBA_routelist():
    url = "http://bustime.mta.info/api/where/routes-for-agency/MTA%20NYCT.json?key=" + API_KEY
    response = requests.get(url, timeout=30)
    routes = response.json()
    return routes


def get_SIRI_request_urlpaths():
    SIRI_request_urlpaths = []
    routes=get_OBA_routelist()
    for route in routes['data']['list']:
        SIRI_request_urlpaths.append({route['id']:"/api/siri/vehicle-monitoring.json?key={}&VehicleMonitoringDetailLevel=calls&LineRef={}".format(API_KEY, route['id'])})
    return SIRI_request_urlpaths


def num_buses(feeds):
    num_buses=0
    for route_report in feeds:
        for route_id,route_data in route_report.items():
            try:
                route_data = route_data.json()
                for monitored_vehicle_journey in route_data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity']:
                    num_buses = num_buses + 1
            except: # no vehicle activity?
                pass
    return num_buses


In [10]:
localhost = True
cwd = '.'
async_grab_and_store(localhost, cwd)

Grabbed 242 route feeds containing 1491 buses in 9.905955 seconds at 2021-09-15 19:58:48.
